In [1]:
import csv
import cv2
import numpy as np
import os

path = '/data/dataset_1'
files_folder = os.listdir(path)

# Read csv file with camera information.
print 'Reading camera.csv file...'
with open(os.path.join(path, 'camera.csv'), 'rb') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    csv_list = list(reader)
    csv_list_camera = csv_list[1:]

# Read inputs from center camera.
center_frames = [f[5] for f in csv_list_camera if 'center_camera' in f]
number_frames = len(center_frames)

# Read csv file from steering.
with open(os.path.join(path, 'steering.csv'), 'rb') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    csv_list = list(reader)
    csv_list_steering = csv_list[1:]
steering_angles = [f[2] for f in csv_list_steering]

#TODO: Use correct labels for steering angle.
steering_angles = steering_angles[:number_frames]
steering_angles = np.array(steering_angles).astype('float32')

# Create 4-dimensional numpy array with images.
# TODO: Check and save tensor in input file
print 'Packing data in a single 4 dimensional array...'
tensor_shape = tuple((0, 480, 640, 3))
tensor_train = np.empty(tensor_shape)
for i, frame in enumerate(center_frames[:50]):
    img_path = os.path.join(path, frame)
    img = cv2.imread(img_path)
    # Add frame into the tensor.
    tensor_train = np.concatenate([tensor_train, img[np.newaxis, :]],
                                  axis=0)

Reading camera.csv file...
Packing data in a single 4 dimensional array...


In [2]:
tensor_train.shape
tensor_shape

(0, 480, 640, 3)

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Convolution2D
from keras.optimizers import SGD
from keras.utils import np_utils


def experiment():
    model = Sequential()

    # 2. ConvLayer: 2x2 stride, 5x5 kernel, 3 kernels.
    model.add(Convolution2D(3, 5, 5,
                            border_mode='same',
                            subsample=(2, 2),
                            input_shape=tensor_shape[1:]))

    # 3. ConvLayer: 2x2 stride, 5x5 kernel, 24 kernels.
    model.add(Convolution2D(24, 5, 5,
                            border_mode='same',
                            subsample=(2, 2)))

    # 4. ConvLayer: 2x2 stride, 5x5 kernel, 36 kernels.
    model.add(Convolution2D(36, 5, 5,
                            border_mode='same',
                            subsample=(2, 2)))

    # 5. ConvLayer: 3x3 kernel, 48 kernels, no stride.
    model.add(Convolution2D(48, 3, 3,
                            border_mode='same'))

    # 6. ConvLayer: 3x3 kernel, 64 kernels, no stride.
    model.add(Convolution2D(64, 3, 3,
                            border_mode='same'))

    # 7. FlattenLayer.
    model.add(Flatten())

    # 8. DenseLayer: 1164
    model.add(Dense(1164))

    # 9. DenseLayer: 100
    model.add(Dense(100))

    # 10. DenseLayer: 50
    model.add(Dense(50))

    # 11. DenseLayer: 10.Provisionally using 10
    model.add(Dense(1))

    # Layer output. For real data change to linear activation
    model.add(Activation('linear'))
    
    # let's train the model using SGD + momentum (how original).
    # TODO: check for values.
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error',
                  optimizer=sgd,
                  metrics=['accuracy'])


    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=True)  # divide each input by its std

    datagen.fit(tensor_train)
    
    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(tensor_train, steering_angles[:50],
                        batch_size=10),
                        samples_per_epoch=20,
                        nb_epoch=100)    

Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5105)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [4]:
%time experiment()

Epoch 1/100
20/20 [==============================] - 1s - loss: nan - acc: 0.0000e+00                      
Epoch 2/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 3/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 4/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 5/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 6/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 7/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 8/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 9/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 10/100
20/20 [==============================] - 0s - loss: nan - acc: 0.0000e+00     
Epoch 11/100
20/20 [==============================] - 0s - loss: nan - a

In [5]:
steering_angles

array([-0.04712389, -0.04537856, -0.04537856, ...,  0.0418879 ,
        0.04363323,  0.04537856], dtype=float32)